In [1]:
# TAKEN FROM https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#forceEdit=true&sandboxMode=true&scrollTo=8A-1LTSH88EE
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


In [2]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:

# this environment has env.reset() and end.step() functions
env = wrap_env(gym.make('CartPole-v0'))
env.reset()
while True:  
    env.render()    
    #your agent goes here
    action = env.action_space.sample()          
    observation, reward, done, info = env.step(action) 
    if done: 
      break;
env.close()            
show_video()

In [0]:
import argparse
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import time

In [0]:
import random
from gym.wrappers import Monitor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import pickle
from collections import deque

In [0]:
class DQNAgent:
    def __init__(self, env):

        # based on the Gym environment, initialize the action & space sizes:

        if isinstance(env.action_space,gym.spaces.box.Box):
            self.action_size = env.action_space.shape[0]
        elif isinstance(env.action_space,gym.spaces.discrete.Discrete):
            self.action_size = env.action_space.n
        else:
            raise Exception('Not supported action environment')

        if isinstance(env.observation_space,gym.spaces.box.Box):
            self.state_size = env.observation_space.shape[0]
        elif isinstance(env.observation_space,gym.spaces.discrete.Discrete):
            self.state_size = env.observation_space.n
        else:
            raise Exception('Not supported state environment')

        self.memory        = deque(maxlen=2000)    # Memory buffer
        self.gamma         = 0.95                   # Discount factor
#         self.epsilon     = 1.0            # Initial epsilon-greedy factor for searching the environment
        self.epsilon       = 0.01           # When you load trained weights epsilon should be small 
        self.epsilon_min   = 0.01           # Minimum epsilon-greedy factor for searching the environment
        self.epsilon_decay = 0.995          # Decay factor in epsilon
        self.learning_rate = 0.001          # Adam Optimizer Learning rate
        self.model         = self.Estimator()       # Neural Network for approximating the environment

    def Estimator(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    def load(self, address):
        self.model.load_weights(address)

    def save(self, address):
        self.model.save_weights(address)

    def Buffer(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def Actor(self, state):
        if np.random.rand() <= self.epsilon:
            return env.action_space.sample()
        act_values = self.model.predict(state)
        if isinstance(env.action_space,gym.spaces.box.Box):
            return (act_values[0])
        elif isinstance(env.action_space,gym.spaces.discrete.Discrete):
            return np.argmax(act_values[0])

    def Replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if done:
                target_value = reward
            else:
                target_value = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            updated_target   = self.model.predict(state)
            if isinstance(env.action_space,gym.spaces.discrete.Discrete):
                updated_target[0][action] = target_value
            else:
                updated_target[0][np.argmax(action)] = target_value    #you may need to change this line for different space-action spaces
            self.model.fit(state, updated_target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [0]:
name  = 'MountainCar-v0'
# name  = 'CartPole-v1'
# name  = 'Assault-ram-v0'
# env   = gym.make(name)

env = wrap_env(gym.make(name))

agent = DQNAgent(env)
# agent.load('saves/{}-dqn.h5'.format(name))
# with open('saves/{}-deque.txt'.format(name), "rb") as fp:
#     data = pickle.loads(fp.read())
# agent.memory.extend(data)
done         = False
batch_size   = 32
sim_runs     = 100
num_steps    = 500
score_vector = []
    
for sim in range(sim_runs):
    state  = env.reset()
    state  = np.reshape(state, [1, agent.state_size])
    #some environments have maximum number of iterations by themselves 
    for step in range(num_steps): 
#             animation
        # env.render()
        action                      = agent.Actor(state)
        next_state, reward, done, _ = env.step(action)
        reward                      = reward if not done else -10
        next_state                  = np.reshape(next_state, [1, agent.state_size])
        agent.Buffer(state, action, reward, next_state, done)
        state                       = next_state
        if done:
            print("simulation: {}/{}, score: {}".format(sim, sim_runs, step))
            score_vector.append(step)
            break
        if len(agent.memory) > batch_size:
            agent.Replay(batch_size)
    if (sim+1) % 50 == 0:
        print(score_vector)
env.close()

simulation: 0/100, score: 199
simulation: 1/100, score: 199
simulation: 2/100, score: 199
simulation: 3/100, score: 199
simulation: 4/100, score: 199
simulation: 5/100, score: 199
simulation: 6/100, score: 199
simulation: 7/100, score: 199
simulation: 8/100, score: 199
simulation: 9/100, score: 199
simulation: 10/100, score: 199
simulation: 11/100, score: 199
simulation: 12/100, score: 199
simulation: 13/100, score: 199
simulation: 14/100, score: 199
simulation: 15/100, score: 199
simulation: 16/100, score: 199
simulation: 17/100, score: 199
simulation: 18/100, score: 199
simulation: 19/100, score: 199
simulation: 20/100, score: 199
simulation: 21/100, score: 199
simulation: 22/100, score: 199
simulation: 23/100, score: 199
simulation: 24/100, score: 199
simulation: 25/100, score: 199
simulation: 26/100, score: 199
simulation: 27/100, score: 199
simulation: 28/100, score: 199
simulation: 29/100, score: 199
simulation: 30/100, score: 199
simulation: 31/100, score: 199
simulation: 32/100

In [0]:
env = wrap_env(gym.make(name))
state  = env.reset()
state  = np.reshape(state, [1, agent.state_size])
#some environments have maximum number of iterations by themselves 
for step in range(num_steps): 
#             animation
    env.render()
    action                      = agent.Actor(state)
    next_state, reward, done, _ = env.step(action)
    reward                      = reward if not done else -10
    next_state                  = np.reshape(next_state, [1, agent.state_size])
    agent.Buffer(state, action, reward, next_state, done)
    state                       = next_state
    if done:
      break
env.close()            
show_video()

In [0]:
# name  = 'MountainCar-v0'
name  = 'CartPole-v1'
# name  = 'Assault-ram-v0'
# env   = gym.make(name)

env = wrap_env(gym.make(name))

agent = DQNAgent(env)
# agent.load('saves/{}-dqn.h5'.format(name))
# with open('saves/{}-deque.txt'.format(name), "rb") as fp:
#     data = pickle.loads(fp.read())
# agent.memory.extend(data)
done         = False
batch_size   = 32
sim_runs     = 100
num_steps    = 500
score_vector = []
    
for sim in range(sim_runs):
    state  = env.reset()
    state  = np.reshape(state, [1, agent.state_size])
    #some environments have maximum number of iterations by themselves 
    for step in range(num_steps): 
#             animation
        # env.render()
        action                      = agent.Actor(state)
        next_state, reward, done, _ = env.step(action)
        reward                      = reward if not done else -10
        next_state                  = np.reshape(next_state, [1, agent.state_size])
        agent.Buffer(state, action, reward, next_state, done)
        state                       = next_state
        if done:
            print("simulation: {}/{}, score: {}".format(sim, sim_runs, step))
            score_vector.append(step)
            break
        if len(agent.memory) > batch_size:
            agent.Replay(batch_size)
    if (sim+1) % 50 == 0:
        print(score_vector)
env.close()

simulation: 0/100, score: 8
simulation: 1/100, score: 11
simulation: 2/100, score: 8
simulation: 3/100, score: 7
simulation: 4/100, score: 10
simulation: 5/100, score: 8
simulation: 6/100, score: 8
simulation: 7/100, score: 14
simulation: 8/100, score: 14
simulation: 9/100, score: 15
simulation: 10/100, score: 38
simulation: 11/100, score: 8
simulation: 12/100, score: 8
simulation: 13/100, score: 8
simulation: 14/100, score: 7
simulation: 15/100, score: 10
simulation: 16/100, score: 9
simulation: 17/100, score: 8
simulation: 18/100, score: 8
simulation: 19/100, score: 9
simulation: 20/100, score: 7
simulation: 21/100, score: 8
simulation: 22/100, score: 8
simulation: 23/100, score: 7
simulation: 24/100, score: 9
simulation: 25/100, score: 8
simulation: 26/100, score: 9
simulation: 27/100, score: 43
simulation: 28/100, score: 100
simulation: 29/100, score: 47
simulation: 30/100, score: 96
simulation: 31/100, score: 74
simulation: 32/100, score: 92
simulation: 33/100, score: 101
simulati

In [0]:
env = wrap_env(gym.make(name))
state  = env.reset()
state  = np.reshape(state, [1, agent.state_size])
#some environments have maximum number of iterations by themselves 
for step in range(num_steps): 
#             animation
    env.render()
    action                      = agent.Actor(state)
    next_state, reward, done, _ = env.step(action)
    reward                      = reward if not done else -10
    next_state                  = np.reshape(next_state, [1, agent.state_size])
    agent.Buffer(state, action, reward, next_state, done)
    state                       = next_state
    if done:
      break
env.close()            
show_video()